In [6]:
from fun_nearealtime_RTM import *
from scipy.optimize import curve_fit
from main_SW_scope_nearealtime import *

# 1. New method to compute gradient descent for COD.

In [ ]:
data_dir = './GOES_data/'

# change dataset for testing, three options: 'COD>20', 'COD>10', 'COD<10'
figlabel = 'COD>10' # 'COD>10', 'COD<10'

sky = 'day'
phase = 'water'

# start
sat = nearealtime_process_satellite_eT(figlabel, phase, data_dir=data_dir, sky=sky)
# Cont is the constant: tau = 1/(Constant - R)
#please see the explain at 1.1, and 1.2

### func to control it

In [16]:
def nearealtime_process_satellite_eT(figlabel, phase, data_dir=None, sky="day"):
    timeofday = "day"
    # 1. Read satellite data
    Sat_dir = Sat_preprocess(data_dir, figlabel, sky, phase, sat='GOES16')
    sat = pd.read_hdf(
        os.path.join(data_dir + Sat_dir,
                        "GOES_{}_BON_radiance_satellite_{}_{}_{}.h5".format(timeofday, phase, figlabel, sky)),
        'df'
    )
    channels = ['C{:02d}'.format(c) for c in range(1, 6 + 1)]
    rtm_columns = [f'rtm_{ch}' for ch in channels]
    for col in rtm_columns:
        sat[col] = 0
    try:
        sat_ref = sat[channels]
    except KeyError:
        sat_ref = sat['Radiance']

    max_iterations = 10
    # test 0.0006 is a little bit large, most converge less than 0.0001
    epsilon = 0.0006  # n * 0,0001 , V_magnitude 1 * 0.01(1%), single SE = 0.0001,
    max_COD, min_COD = 50, 0

    # 2. Read satellite date and site measurements
    # we only ues the first 3 row as test
    for i in range(3): #sat_ref.shape[0]):
        print("--" * 20)
        print(f'Column {i} of {sat_ref.shape[0]}')
        print(f"{'Iteration':<10} {'COD':<10} {'RMSE':<10}{'Cost':<10}{'Cont':<10}")
        Sun_Zen = sat['Sun_Zen'].iloc[i]
        T_a = sat['temp'].iloc[i] + 273.15
        RH = sat['rh'].iloc[i]
        local_zen = sat.get('Local_Ze', sat.get('local_Zen')).iloc[i]
        rela_azi = sat['rela_azi'].iloc[i]
        Rc_real = min_max_nor(sat_ref.iloc[i].to_frame().T)  # sat_ref.iloc[i], sys=sys)

        Rc_rtm_df = pd.DataFrame()
        Rc_rtm_rad_df = pd.DataFrame()
        RMSE_dic, COD_list = [], []
        Costs = []

        # 3. Gradient descent
        COD_guess = 10 # initial guess
        C = 1 # initial value for the cost function
        for j in range(max_iterations):
            # once COD_guess is out of range, exit the loop
            if COD_guess > max_COD or COD_guess < min_COD or j == max_iterations - 1:
                COD_guess = COD_list[np.argmin(RMSE_dic)]
                sat.at[i, 'COD_rtm_6c'] = COD_guess
                break
            COD_list.append(COD_guess)
            # calculate radiance using nearealtime_LUT
            Rc_rtm_radiance = nearealtime_LUT(Sun_Zen, local_zen, rela_azi, COD_guess, T_a, RH,
                                              file_dir=data_dir, bandmode='GOES')
            Rc_rtm_rad_df = pd.concat([Rc_rtm_rad_df, Rc_rtm_radiance], ignore_index=True)
            Rc_rtm = min_max_nor(Rc_rtm_radiance)
            Rc_rtm_df = pd.concat([Rc_rtm_df, Rc_rtm], ignore_index=True)

            SE, MSE, RMSE = loss_function(Rc_real.values[0], Rc_rtm.values[0])
            cost = MSE 
            RMSE_dic.append(RMSE)
            Costs.append(cost)

            print(f"Iter {j:<5} {COD_guess:<10.2f}{RMSE:<10.4f}{cost:<10.4f}{C:10.4f}")
            if cost < epsilon:
                sat.at[i, 'COD_rtm_6c'] = COD_guess
                break
            COD_guess, C = compute_gradient_eT(Rc_rtm_df, COD_list,Rc_real, Costs, C)
        print(f"Final COD: {COD_guess:.4f}")
        for channel in channels:
            rtm_channel = 'rtm_' + channel
            sat.at[i, rtm_channel] = Rc_rtm_rad_df[channel].values[-1]
    return sat

### 1.1 thoery for the compute_gradient_eT
#### Gradient Descent to estimiate COD
We supposed the relationship between satellite R and $\tau$ is:
$R = C - e^{-\tau}$

We guess the value of $\tau$ using gradient descent method, and the gradient is calculated as follows:
$$ 
\tau_{guess} = \tau_0 + \frac{d\tau}{dR}\Delta{R} \tag{1}
$$
in which the derivative is:

$$
\frac{d\tau}{dR} = \frac{1}{C-R} \tag{2}
$$

In [5]:
def compute_gradient_eT(Rc_rtm_df, COD_list, Rc_real, Costs, C, EDU=True):
    """
    Linear fitting of COD, based on two nearest neighbors of Rc_real.
    """
    n = Rc_rtm_df.shape[1]  # number
    sorted_indices = np.argsort(Costs)  # ascendent order
    nearest_indices = sorted_indices[:2]
    # Sort nearest indices to ensure consistent order
    
    Rc0 = Rc_rtm_df.iloc[nearest_indices[0]]
    COD0 = COD_list[nearest_indices[0]]

    # Euclidean distance
    if EDU == True:
        f1 = 1 / (C - np.linalg.norm(Rc0)) # eq(2)
        dx = np.linalg.norm(Rc_real.values - Rc_rtm_df.iloc[nearest_indices[0]].values)
    else:
        f1 = 1 / (C - Rc0.sum())
        dx = (Rc_real.values - Rc_rtm_df.iloc[nearest_indices[0]].values).sum(axis=1)
        
    COD_guess = COD0 + f1 * dx  # eq(1)
    print('point1', COD0, 'COD_guess', float(COD_guess),'f1',f1,'dx',dx)

    # learning rate
    lr = 10  # lr is estimated by number of COD_list
    dj_dw = gradient_function(COD_list, Rc_rtm_df, C) # eq4
    C = C - lr * 0.5 * dj_dw / n  # Adjust w to avoid extrapolation
    return COD_guess, C

### 1.2 thoery for gradient_function
####Minimum Cost function J to update C
We want the cost J very small for 6 channels.
$$J(C) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f(\tau^{(i)}) - R_{sat}^{(i)})^2\tag{3}$$
where $f(\tau^{(i)})$ is $R_i = C - e^{-\tau_i} $

the C is not 1, we update it each time.

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \newline
\; C &= C - \alpha \frac{\partial J(C)}{\partial C} \tag{3} \; \newline
\frac{\partial J(C)}{\partial C} &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f(\tau^{(i)}) - R_{sat}^{(i)}) \tag{4} \; \newline
\end{align*}$$

where $\alpha$ is the learning rate=1, and $m$ is the number of channels.

In [14]:
# eq(4)
def gradient_function(x, y_sat, C):
    """
    Cost = 1/m * sum(  f(x_i) - y_i_sat)^2  )
    f(x_i) = C - exp(-x_i)
    df(x_i)_dC = 1
    dCost_dC = 1/m * sum( f(x_i) - y_i_sat) )

    Parameters
    ----------
    x : 1D COD
    y_sat : 2D pd.DataFrame, 6 channels
    C : float scalar

    Returns
    -------

    """
    diff = 0
    for i in range(len(x)):
        xi = x[i]
        yi_sat = y_sat.iloc[i].values
        y_pred = C-np.exp(-xi)
        diff += np.linalg.norm(yi_sat) - y_pred
    dj_dw = diff/ len(x)  # Average gradient
    return dj_dw

# 2. Linear or Exponential interpolation or extrapolation

In [ ]:
sat = nearealtime_process_satellite(figlabel, phase, data_dir=data_dir, sky=sky)

### functions 

In [ ]:
def nearealtime_process_satellite(figlabel, phase, data_dir=None, sky="day", N_bundles = 10000):
    if sky == "night":
        timeofday = "night"
    else:
        timeofday = "day"
    Sat_dir = Sat_preprocess(data_dir, figlabel, sky, phase, sat='GOES16')
    if sky=='clearsky':
        sat = pd.read_hdf(
        os.path.join(data_dir + Sat_dir,
                     "GOES_{}_BON_radiance_satellite_{}_{}.h5".format(timeofday, figlabel, sky)),
                    'df'
                    )
    else:
        sat = pd.read_hdf(
        os.path.join(data_dir + Sat_dir,
                     "GOES_{}_BON_radiance_satellite_{}_{}_{}.h5".format(timeofday, phase, figlabel, sky)),
                    'df'
                    )
    channels = ['C{:02d}'.format(c) for c in range(1, 6 + 1)]

    rtm_columns = [f'rtm_{ch}' for ch in channels]
    for col in rtm_columns:
        sat[col] = 0
    try:
        sat_ref = sat[channels]
    except KeyError:
        sat_ref = sat['Radiance']
    
    max_iterations = 10
    # test 0.0006 is a little bit large, most converge less than 0.0001
    #epsilon = 0.0001  # n * 0,0001 , V_magnitude 1 * 0.01(1%), single SE = 0.0001,
    epsilon = 0.0001  # 0.05^2  (5% error for each chnn)
    max_COD, min_COD = 50, 0

    for i in range(sat_ref.shape[0]):
        print("--" * 20)
        print(f'Column {i} of {sat_ref.shape[0]}')
        print(f"{'Iteration':<10} {'COD':<10} {'MSE':<10} {'RMSE':<10} {'Cost':<10}")
        Sun_Zen = sat['Sun_Zen'].iloc[i]
        T_a = sat['temp'].iloc[i] + 273.15
        RH = sat['rh'].iloc[i]
        local_zen = sat.get('Local_Ze', sat.get('local_Zen')).iloc[i]
        rela_azi = sat['rela_azi'].iloc[i]
        Rc_real = min_max_nor(sat_ref.iloc[i].to_frame().T)  # sat_ref.iloc[i], sys=sys)

        Rc_rtm_df = pd.DataFrame()
        Rc_rtm_rad_df = pd.DataFrame()
        RMSE_dic, COD_list = [], [10, 20]
        Costs = []

        # 3. Gradient descent. !!!!!!!!!!!!!!!  from here is the difference.
        # setp 1, inital guess, 10 and 20
        for COD_guess in COD_list:
            Rc_rtm_radiance = nearealtime_LUT(Sun_Zen, local_zen, rela_azi, COD_guess, T_a, RH,
                                              file_dir=data_dir, bandmode='GOES')
            Rc_rtm_rad_df = pd.concat([Rc_rtm_rad_df, Rc_rtm_radiance], ignore_index=True)

            Rc_rtm = min_max_nor(Rc_rtm_radiance)
            Rc_rtm_df = pd.concat([Rc_rtm_df, Rc_rtm], ignore_index=True)
            SE, MSE, RMSE = loss_function(Rc_real.values[0], Rc_rtm.values[0])
            if COD_guess==10 and np.sum(Rc_real.values - Rc_rtm.values) < 0: # for COD<10 cases.
                COD_list[-1] = 0

            cost = np.sum(SE) / 12  # 2*n_channels=12
            RMSE_dic.append(RMSE)
            Costs.append(cost)

            print(f"{'--':<10} {COD_guess:<10.2f}{MSE:<10.4f} {RMSE:<10.4f} {cost:<10.4f}")
            if cost < epsilon:
                sat.at[i, 'COD_rtm_6c'] = COD_guess
                break
        else:  # for else loop
            EUD = True
            # step 2. Use gradient descent
            COD_guess = compute_gradient_simple(Rc_rtm_df, COD_list,
                                                Rc_real, Costs, EUD)
            for j in range(max_iterations):
                if COD_guess > max_COD or COD_guess < min_COD or j == max_iterations-1:
                    COD_guess = COD_list[np.argmin(RMSE_dic)]
                    sat.at[i, 'COD_rtm_6c'] = COD_guess
                    break
                COD_list.append(COD_guess)
                Rc_rtm_radiance = nearealtime_LUT(Sun_Zen, local_zen, rela_azi, COD_guess, T_a, RH,
                                                  file_dir=data_dir, bandmode='GOES')

                Rc_rtm_rad_df = pd.concat([Rc_rtm_rad_df, Rc_rtm_radiance], ignore_index=True)

                Rc_rtm = min_max_nor(Rc_rtm_radiance)
                Rc_rtm_df = pd.concat([Rc_rtm_df, Rc_rtm], ignore_index=True)
                SE, MSE, RMSE = loss_function(Rc_real.values[0], Rc_rtm.values[0])
                cost = np.sum(SE) / 12
                RMSE_dic.append(RMSE)
                Costs.append(cost)
                print(f"Iter {j:<5} {COD_guess:<10.2f}{MSE:<10.4f} {RMSE:<10.4f} {cost:<10.4f}")
                if cost < epsilon:
                    sat.at[i, 'COD_rtm_6c'] = COD_guess
                    break
                COD_guess = compute_gradient_simple(Rc_rtm_df, COD_list,
                                                    Rc_real, Costs)
                

        print(f"Final COD: {COD_guess:.4f}")
        for channel in channels:
            rtm_channel = 'rtm_'+channel
            sat.at[i, rtm_channel] = Rc_rtm_rad_df[channel].values[-1]
    return sat

In [5]:
def compute_gradient_simple(Rc_rtm_df, COD_list, Rc_real, Costs, EDU=True):
    """
    Linear fitting of COD, based on two nearest neighbors of Rc_real.
    1. if COD_guess between COD values, we have already saved in COD_list: interpolation, default. stop line 28, then return.
    2. if COD_guess outside COD values, we use extrapolation.
           if extrapolation, we use exponential function to fit COD = R**3
           R**3 not work, we use gradient descent to update w. [see Andrew Ng's note below.]
    """
    n = Rc_rtm_df.shape[1]  # number
    sorted_indices = np.argsort(Costs)  # ascendent order
    nearest_indices = sorted_indices[:2]
    # Sort nearest indices to ensure consistent order
    # nearest_indices.sort()
    Rc1, Rc2 = Rc_rtm_df.iloc[nearest_indices[0]], Rc_rtm_df.iloc[nearest_indices[1]]
    COD1, COD2 = COD_list[nearest_indices[0]], COD_list[nearest_indices[1]]
    #eT = np.exp(-COD2)
    # Compute the gradient using the two nearest points
    ## method 1 : take w as the comprehensive gradient for comprehensive X
    if EDU == True:
        w = (COD2 - COD1) / np.linalg.norm(Rc2 - Rc1)  # np.sum(Rc2 - Rc1)
        w = 1 / np.linalg.norm(Rc2)
        dx = np.linalg.norm(Rc_real.values - Rc_rtm_df.iloc[nearest_indices[0]].values)
    else:
        w = (COD2 - COD1) / (Rc2 - Rc1).sum()
        dx = (Rc_real.values - Rc_rtm_df.iloc[nearest_indices[0]].values).sum(axis=1)

    COD_guess = COD1 + w * dx  # the positive and negative cancelled
    print('point1', COD1, 'point2', COD2, 'COD_guess', float(COD_guess))

    # Extrapolation check
    max_COD = max(COD1, COD2)
    # avoid repeat value
    condition = np.abs(np.array(COD_list) - COD_guess) <= 1
    # Check extrapolation condition
    is_extrapolation = COD_guess > max_COD or (np.sum(condition) >= 1 and len(COD_list) > 2)
    
    if is_extrapolation:
        print("Extrapolation triggered")
        x = Rc_rtm_df.sum(axis=1).values # distance from the 0 vector
        y = np.array(COD_list)
        try:
            # using exponential function y = x**3 to fit COD = R**3.
            popt, _ = curve_fit(expol_func, x, y, bounds=(1e-8, 10)) # bound for a in expol_func
            a_fit = popt[0]
            x_r = Rc_real.sum(axis=1).values
            COD_guess = expol_func(x_r, a_fit)
            condition = np.abs(np.array(COD_list) - COD_guess) <= 1
            print('Extrap triggered COD = ', COD_guess)

            # if COD increase slowly, was gradient descent to update w.
            if COD_guess>10 and np.sum(condition) >= 2: # COD>20, when testing. 
                lr = len(COD_list)  # lr is estimated by number of COD_list
                print(f"Exponential fitted step too slow for 2 times, lr = {lr} is used")
                dj_dw = np.sum(Rc_rtm_df.iloc[nearest_indices[0]].values - Rc_real.values) * COD1 # eq(4)
                w = w - lr * 0.5 * dj_dw / n  # Adjust w to avoid extrapolation # eq(3)
                # w = w -lr * dj_dw /2*n is the defination of GD.
                dx = np.sum(Rc_real.values - Rc_rtm_df.iloc[nearest_indices[0]].values)
                COD_guess = COD1 + w * dx  # fallback
                print('Condition 2 COD = ', COD_guess)
        except Exception as e:
            # the same as above, not using expol_func
            lr = len(COD_list)  # lr is estimated by number of COD_list
            print(f"Exponential fit failed: {e}, lr = {lr} is used")
            dj_dw = np.sum(Rc_rtm_df.iloc[nearest_indices[0]].values - Rc_real.values) * COD1 # eq(4)
            w = w - lr * 0.5 * dj_dw / n  # Adjust w to avoid extrapolation # eq(3)
            # w = w -lr *dj_dw /2*n is the defination of GD.
            dx = np.sum(Rc_real.values - Rc_rtm_df.iloc[nearest_indices[0]].values)
            COD_guess = COD1 + w * dx  # fallback
            print('Exception COD = ',COD_guess)
    return float(COD_guess)

<a name="toc_40291_2.1"></a>
## Gradient descent summary
So far in this course, you have developed a linear model that predicts $f_{w,b}(x^{(i)})$:
$$f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$
In linear regression, you utilize input training data to fit the parameters $w$,$b$ by minimizing a measure of the error between our predictions $f_{w,b}(x^{(i)})$ and the actual data $y^{(i)}$. The measure is called the $cost$, $J(w,b)$. In training you measure the cost over all of our training samples $x^{(i)},y^{(i)}$
$$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2\tag{2}$$ 


In lecture, *gradient descent* was described as:

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline
\;  w &= w -  \alpha \frac{\partial J(w,b)}{\partial w} \tag{3}  \; \newline 
 b &= b -  \alpha \frac{\partial J(w,b)}{\partial b}  \newline \rbrace
\end{align*}$$
where, parameters $w$, $b$ are updated simultaneously.  
The gradient is defined as:
$$
\begin{align}
\frac{\partial J(w,b)}{\partial w}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)} \tag{4}\\
  \frac{\partial J(w,b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \tag{5}\\
\end{align}
$$

Here *simultaniously* means that you calculate the partial derivatives for all the parameters before updating any of the parameters.